In [5]:
import boto3
import time
import logging
import pprint

In [3]:
agent_client = boto3.client('bedrock-agent')
agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [6]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [9]:
def invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False, session_state=None):
    end_session: bool = False
    if not session_state:
        session_state = {}

    # invoke the agent API
    agent_response = agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=enable_trace,
        endSession=end_session,
        sessionState=session_state
    )

    if enable_trace:
        logger.info(pprint.pprint(agent_response))

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [7]:
import uuid
session_id:str = str(uuid.uuid1())


In [12]:
%%time

agent_id = "FDINTOZLTQ"
alias_id = "K0F2DLTPSM"
query = "What is on the adult menu?"

response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

The adult menu at The Regrettable Experience includes:


- Soup of the Day options like Black Bean Soup, New England Clam Chowder, and Hearty Lentil Soup
- Salad options like Southwest Chicken Salad, Grilled Chicken Caesar Salad, Strawberry Spinach Salad, and Greek Salad
- Sandwich options like Turkey Bacon Avocado Melt and BLT Wrap with Avocado
- Entree options like BBQ Pulled Pork Sandwiches, Steak with Garlic Butter, and Vegetable Stir-Fry with Tofu
- Dessert options like Classic New York Cheesecake and Apple Pie a la Mode

CPU times: user 22.5 ms, sys: 837 μs, total: 23.4 ms
Wall time: 6.46 s
